<a href="https://cognitiveclass.ai "><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in TORONTO</font></h1>

Bamou


In [ ]:
import numpy as np
import pandas as pd

The next task is essentially scarping data the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, using pandas method pd.read_html.
The output will be the dataframe table that contains three columns 'Postcode',	'Borough' and	'Neighbourhood' 

In [ ]:
main_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_tables=pd.read_html(main_url)

In [4]:
len(df_tables)

3

In [5]:
print('The function has returned {} tables'.format(len(df_tables)))

The function has returned 3 tables


In [6]:
df_tables[0]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [ ]:
df1=df_tables[0] 

## Data wrangling
### To create the target dataframe, we will ensure that:

1. The dataframe will consist of three columns: PostalCode, 
Borough, and Neighborhood
2. row with 'Not assigned' borough will be deleted
3. More than one neighborhood can exist in one postal code area. 
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 


The next task will delete not assigned boroughs. The resulting dataframe will have 210 rows since there are 77 rows with not assigned boroughs.

In [8]:
df1_filter=pd.DataFrame()
for i in range(df1.shape[0]):
  if df1.iloc[i,1] != 'Not assigned':
    df1_filter=pd.concat([df1_filter, df1.iloc[[i],:]], axis=0)
df1_filter

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


The next task is to filter all the postalcode in the dataframe and put them in an array to be able to iterate on postalcode elements.

In [9]:
ndf1= df1_filter.Postcode.unique()
ndf1

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

The following code enable us to visualize the structure of the dataframe grouped by PostalCode. 

In [10]:
#visualize the struture of the dataframe when grouping by Postcode
df1_filter.groupby('Postcode').count().reset_index()

,Postcode,Borough,Neighbourhood
0,M1B,2,2
1,M1C,3,3
2,M1E,3,3
3,M1G,1,1
4,M1H,1,1
...,...,...,...
98,M9N,1,1
99,M9P,1,1
100,M9R,4,4
101,M9V,8,8


The following tasks give us some visualizations to understand the data structure, and do some necessary manipulation.

In [ ]:
# applying groupby() function to group the data on Postcode value.
gf=df1_filter.groupby('Postcode')

In [12]:
#visualize a sub dataframe where 'Postcode'='M8Z'
gff=gf.get_group('M8Z')
gff

,Postcode,Borough,Neighbourhood
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [13]:
#Transform the Neighbourhoud serie to a multiline string then manipulate the string to fit in one line separated by comma
sff= gff.Neighbourhood.to_string(index=False)
sff=sff.replace('\n  ', ",")
print(sff)



 Kingsway Park South West,              Mimico NW,     The Queensway West,  Royal York South West,         South of Bloor


In [14]:
#Transform the column "Borough" to string, then split the string and keep the first element of the list
bff=gff.Borough.to_string(index=False)
bff=bff.replace('\n', ',').strip()
bff=bff.split(',')[0]
print('The Borough is {}'.format(bff))


The Borough is Etobicoke


With the understanding of data structure, the next task will transform the dataframe and respond to the three initial requirements:


In [15]:
df2_filter=pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood']) #initialize empty dataframe with columns

for M in ndf1:
  gff=gf.get_group(M)

#Transform data in Borough to get only one occurence
  bff=gff.Borough.to_string(index=False)
  bff=bff.replace('\n', ',').strip()
  bff=bff.split(',')[0]

#Transform data in neighbourhood to fit in a single row
  nff= gff.Neighbourhood.to_string(index=False)
  nff=nff.replace('\n', ",")
  

  values = [M, bff, nff] # define values
  df2_filter.loc[len(df2_filter.index)+1] = values # add values to row at end of dataframe

df2_filter.head()

,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


Now, we will replace the name of the first column from 'Postcode' to 'PostalCode'

In [16]:
df_tor=df2_filter
for i in range(df_tor.shape[0]):
  a=df2_filter.iloc[i,2].strip()
  if a == 'Not assigned':
    df_tor.iloc[i,2]=df_tor.iloc[i,1]

df_tor.rename(columns={'Postcode':'PostalCode'}, inplace=True)    
df_tor.head(10) 


,PostalCode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,"Rouge, Malvern"
8,M3B,North York,Don Mills North
9,M4B,East York,"Woodbine Gardens, Parkview Hill"
10,M5B,Downtown Toronto,"Ryerson, Garden District"


The final task of this notebook is to print the number of rows of the dataframe using the function shape.

In [17]:
df_tor.shape
print('The number of rows in the dataframe is {}'.format(df_tor.shape[0]))

The number of rows in the dataframe is 103


Now that the dataframe is clean, and in the right format we have to save it for furher use.
The do that the clean dataframe will be transformed and saved as csv file.

In [ ]:
from google.colab import files
df_tor.to_csv('result1.csv')
files.download('result1.csv') 